# **Final Project: Khai thác và trực quan hóa dữ liệu**

### **Danh sách các thành viên**

<center>

|         Họ tên         |   MSSV   |
| :--------------------- | :------: |
| Trần Nguyễn Nhật Cường | 22127048 |
|    Nguyễn Công Tuấn    | 22127436 |
|     Trần Đăng Tuấn     | 22127438 |
| ... | ... |
| ... | ... |

</center>

## **Cài đặt và sử dụng các thư viện cần thiết**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests

import copy

**`matplotlib`** được dùng để vẽ biểu đồ trong Python. Có thể dùng để vẽ các loại biểu đồ như
histogram, biểu đồ tán xạ (scatter plot), biểu đồ đường (line chart) và biểu đồ cột (bar chart),...

**`pandas`** được dùng để xử lý và phân tích dữ liệu dạng bảng và cung cấp cấu trúc dữ liệu như
DataFrame và Series để dễ thao tác.

**`seaborn`** dùng để vẽ biểu đồ dựa trên thư viện matplotlib nhưng biểu đồ sẽ trông đẹp hơn và cũng hỗ trợ các loại biểu đồ thống kê như violin plot, box plot, heatmap.

**`numpy`** được dùng để xử lý các phép toán số học nhanh trên mảng nhiều chiều (arrays) và hỗ trợ các phép toán ma trận, thống kê, đại số tuyến tính cần thiết cho phân tích dữ liệu.


**`requests`** được dùng để gửi các yêu cầu HTTP (như GET, POST) nhằm truy vấn dữ liệu từ các API hoặc tải dữ liệu từ Internet về máy tính.

## **Thu thập dữ liệu**

In [ ]:
offset = 0
limit = 1000
all_records = []

while True:
    url = f'https://data.vietnam.opendevelopmentmekong.net/en/api/3/action/datastore_search'
    params = {
        'resource_id': 'a03fb5c6-4622-4196-a926-15e9542c4ae8',
        'limit': limit,
        'offset': offset
    }
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        data = response.json()
        records = data['result']['records']
        
        if not records:
            break
        
        all_records.extend(records)
        offset += limit
    else:
        break

DATA = pd.DataFrame(all_records)


In [ ]:
data = copy.deepcopy(DATA)

In [ ]:
data.head()

## **Khám phá dữ liệu**

<div align='center'>

| Cột | Mô tả | 
| --- | --- |
| `_id` | 	ID tự động của hệ thống để định danh từng hàng trong cơ sở dữ liệu |
| `ID` | Mã số định danh riêng của từng khu đất/lô đất |
| `Latitude` | 	Vĩ độ (tọa độ địa lý) của khu đất/lô đất |
| `Longtitude` | Kinh độ (tọa độ địa lý) của khu đất/lô đất. |
| `Province` | Tên tỉnh/thành phố nơi khu đất/lô đất tọa lạc. |
| `District` | Tên quận/huyện nơi khu đất/lô đất tọa lạc. |
| `Commune` | Tên xã/phường/thị trấn nơi khu đất/lô đất tọa lạc. |
| `Sub-regional` | 	Khu vực/tiểu vùng nhỏ hơn trong xã hoặc huyện (thường là cụm nhỏ hơn). |
| `Plot` | Số hiệu thửa đất (Plot number). |
| `Lot` | Số hiệu lô đất trong thửa đất. |
| `Site` | Tên cụ thể của khu vực/quần thể đất (ví dụ: tên thôn, làng). |
| `Forest owner	` | Tên chủ sở hữu hoặc đơn vị quản lý rừng. |
| `Land type` | Loại hình đất tại khu vực (ví dụ: rừng tự nhiên trung bình, đất rừng trồng...). |
| `Original resources of forest` | Loại tài nguyên rừng ban đầu (ví dụ: rừng tự nhiên, rừng trồng...). |
| `Date` | Ngày ghi nhận hoặc cập nhật thông tin khu đất/lô đất. |
| `Priority` | Mức độ ưu tiên quản lý hoặc bảo vệ khu đất/lô đất (ví dụ: High - Cao). | 
| `Contractor` | Tên nhà thầu hoặc cá nhân phụ trách khai thác hoặc trồng rừng (nếu có). |
| `Status` | Tình trạng hiện tại của khu đất/lô đất (ví dụ: Verified - Đã xác minh). |

</div>

In [ ]:
data.info()

**Những thuộc tính có chứa các hàng bị thiếu dữ liệu** 

In [ ]:
data.replace("", np.nan, inplace=True)

null_counts = data.isnull().sum()

for col, num_nulls in null_counts.items():
    if num_nulls > 0:
        print(f"{col} has '{num_nulls}' null records ")

**Nhận xét** - Vậy các thuộc tính `Commune`, `Sub-regional`, `Plot`, `Lot`, `Site`, `Forest owner`, `Land type`, `Date` và `Contractor`

**Kiểm tra có dữ liệu trùng lặp hay không**

In [ ]:
print('Số lượng các hàng bị trùng lặp là: ', data.duplicated().sum())

## **Tiền xử lý dữ liệu**

**Xử lý dữ liệu bị mất**

**Xử lý cột Commune**

Điền vào các hàng chứa giá trị của thuộc tính `Commune` bị mất bằng cách gom nhóm `Province` và `District` và sau đó xem liệu `Commune` bị mất đó thuộc nhóm `Province` và `District` nào

**Xử lý cột Sub-regional**

Điền vào các hàng chứa giá trị của thuộc tính `Sub-regional` bị mất bằng cách gom nhóm `Province`, `District` và `Commune` và sau đó xem liệu `Sub-regional` bị mất đó thuộc nhóm `Province`, `District` và `Commune`


In [ ]:
data['Commune'] = data.groupby(['Province', 'District'])['Commune'].transform(lambda x: x.ffill().bfill())

data['Sub-regional'] = data.groupby(['Province', 'District', 'Commune'])['Sub-regional'].transform(lambda x: x.ffill().bfill())

**Xử lý cột Plot**

Kiểm tra xem liệu giá trị của cột `Plot` có phải là duy nhất (unique) trong nhóm `Province`, `District`, `Commune` và `Sub-regional` không?

In [ ]:
grouped_plot = data.groupby(['Province', 'District', 'Commune', 'Sub-regional'])['Plot'].unique()

for group, plots in grouped_plot.items():
    print(f"Nhóm {group} có plot: {plots}")

**Nhận xét** - Ta thấy rằng giá trị của cột `Plot` không phải là duy nhất (unique) trong nhóm `Province`, `District`, `Commune` và `Sub-regional`


Cách xử lý thay thế: Xem các hàng chứa thuộc tính `Plot` bị mất gần với `Longtitude` và `Latitude` nhất trong cùng một nhóm `Province`, `District`, `Commune` và `Sub-regional`. Nhưng nếu nhóm `Province`, `District`, `Commune` và `Sub-regional` không có `Plot` nào thì sẽ mặc định điền là 0

In [ ]:
groups = data.groupby(['Province', 'District', 'Commune', 'Sub-regional'])

for name, group in groups:
    missing_plot = group[group['Plot'].isnull()]
    available_plot = group[group['Plot'].notnull()]
    
    if not missing_plot.empty and not available_plot.empty:
        for idx_missing, row_missing in missing_plot.iterrows():
            lon_lat_missing = np.array([row_missing['Longtitude'], row_missing['Latitude']])
            lon_lat_available = available_plot[['Longtitude', 'Latitude']].values

            distances = np.sqrt(np.sum((lon_lat_available - lon_lat_missing)**2, axis=1))
            
            nearest_plot = available_plot.iloc[distances.argmin()]['Plot']
            
            data.at[idx_missing, 'Plot'] = nearest_plot
    elif available_plot.empty:
        data.loc[missing_plot.index, 'Plot'] = 0

**Xử lý cột Lot**

Kiểm tra xem liệu giá trị của cột `Lot` có phải là duy nhất (unique) trong nhóm `Province`, `District`, `Commune`, `Sub-regional` và `Plot` không?

In [ ]:
grouped_lot = data.groupby(['Province', 'District', 'Commune', 'Sub-regional', 'Plot'])['Lot'].unique()

for group, lots in grouped_lot.items():
    print(f"Nhóm {group} có lot: {lots}")

**Nhận xét** - Ta thấy rằng giá trị của cột `Lot` không phải là duy nhất (unique) trong nhóm `Province`, `District`, `Commune`, `Sub-regional` và `Plot`


Cách xử lý thay thế: Xem các hàng chứa thuộc tính `Lot` bị mất gần với `Longtitude` và `Latitude` nhất trong cùng một nhóm `Province`, `District`, `Commune`, `Sub-regional` và `Plot`. Nhưng nếu nhóm `Province`, `District`, `Commune`, `Sub-regional` và `Plot` không có `Lot` nào thì sẽ mặc định điền là 0

In [ ]:
groups = data.groupby(['Province', 'District', 'Commune', 'Sub-regional', 'Plot'])

for name, group in groups:
    missing_plot = group[group['Lot'].isnull()]
    available_plot = group[group['Lot'].notnull()]
    
    if not missing_plot.empty and not available_plot.empty:
        for idx_missing, row_missing in missing_plot.iterrows():
            lon_lat_missing = np.array([row_missing['Longtitude'], row_missing['Latitude']])
            lon_lat_available = available_plot[['Longtitude', 'Latitude']].values

            distances = np.sqrt(np.sum((lon_lat_available - lon_lat_missing)**2, axis=1))
            
            nearest_plot = available_plot.iloc[distances.argmin()]['Lot']
            
            data.at[idx_missing, 'Lot'] = nearest_plot
    elif available_plot.empty:
        data.loc[missing_plot.index, 'Lot'] = 0

**Xử lý cột Site**

Vì `Site` là tên cụ thể của khu vực/quần thể đất (ví dụ: tên thôn, làng) nên nếu nó không có thì có thể là nó chưa được người dân tại nơi đây đặt tên. Vì thế chúng ta sẽ điền vào các hàng có chứa `Site` bị thiếu là **Unknown**

In [ ]:
data['Site'] = data['Site'].fillna('Unknown')

**Xử lý cột Forest owner**

Vì `Forest owner` là tên chủ sở hữu hoặc đơn vị quản lý rừng. nên nếu nó không có thì tức là nó đang không được sở hữu bởi bất kì cá nhân, tổ chức hoặc đơn vị quản lý rừng nào. Vì thế chúng ta sẽ đinef vào các hàng có chứa `Forest owner` bị thiếu là **Unknown**

In [ ]:
data['Forest owner'] = data['Forest owner'].fillna('Unknown')

**Xử lý cột Land type**

Những hàng nào có giá trị bị thiếu ở cột `Land type` sẽ được điền vào bởi những hàng có cùng giá trị nhóm `Province`, `District`, `Commune`, `Sub-regional` và `Plot`

In [ ]:
groups = data.groupby(['Province', 'District', 'Commune', 'Sub-regional', 'Plot'])

for name, group in groups:
    missing_landtype = group[group['Land type'].isnull()]
    available_landtype = group[group['Land type'].notnull()]
    
    if not missing_landtype.empty and not available_landtype.empty:
        landtype_value = available_landtype.iloc[0]['Land type']
        
        data.loc[missing_landtype.index, 'Land type'] = landtype_value

**Xử lý cột Date**

In [ ]:
print('Tỉ lệ giá trị bị mất (null) của cột Contractor so với bộ dữ liệu: ', round(data['Date'].isnull().sum() / data.shape[0], 4))

Do tỉ lệ giá trị bị mất (null) của cột `Date` là quá nhỏ so với toàn bộ dữ liệu nên ta sẽ bỏ hàng bị thiếu giá trị của thuộc tính `Date`

In [ ]:
data.drop(data[data['Date'].isnull()].index, inplace=True)

**Xử lý cột Contractor**

In [ ]:
print('Tỉ lệ giá trị bị mất (null) của cột Contractor so với bộ dữ liệu: ', data['Contractor'].isnull().sum() / data.shape[0])

Do tỉ lệ giá trị bị mất (null) của cột `Contractor` là quá lớn so với toàn bộ dữ liệu nên ta sẽ bỏ cột `Contractor`

In [ ]:
data.drop(columns=['Contractor'], inplace=True)

**Kết quả sau khi xử lý các giá trị bị mất**

In [ ]:
null_counts = data.isnull().sum()

for col, num_nulls in null_counts.items():
    if num_nulls > 0:
        print(f"{col} has '{num_nulls}' null records ")